[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeffheaton/bitphonic/blob/main/bitphonic.ipynb)

# Bitphonic

Copyright 2025 by [Jeff Heaton](https://www.youtube.com/channel/UCR1-GEpyOPzT2AO4D_eifdw), [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0)

This is a fully custom polyphonic subtractive synthesizer written in just over 200 lines of Python. It generates music entirely from math: oscillators (sine, square, triangle, sawtooth, and noise) are shaped by per-note ADSR envelopes, filtered with a Moog-style resonant low-pass filter, and optionally modulated with LFOs for pitch or amplitude effects. Voices are defined in a separate JSON configuration file, allowing each track to use its own multi-oscillator patch. A sequence of notes, extracted from a MIDI files, is rendered sample by sample into a final .WAV audio file. There are no external plugins or sample libraries; all sound comes from math, memory, and code.

# Install Needed Files

We make use of pretty_midi to load in MIDI files to obtain notes to play. MIDI is not used to actually play the files.

In [1]:
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.1 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592286 sha256=a6ab00060ed9142023e77ce725857679b7a8a7c0aa0978ec5beb969ea91c6c14
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi


These two files will be helpful to get started. The 'instruments.json' contains some synth instruments I created to get started. I also provide the notes for [popcorn](https://en.wikipedia.org/wiki/Popcorn_(instrumental)).

In [2]:
!wget -O instruments.json https://s3.us-east-1.amazonaws.com/data.heatonresearch.com/bitphonic/instruments.json
!wget -O Popcorn.json https://s3.us-east-1.amazonaws.com/data.heatonresearch.com/bitphonic/Popcorn.json

PATH = "/content/"

--2025-06-30 04:14:47--  https://s3.us-east-1.amazonaws.com/data.heatonresearch.com/bitphonic/instruments.json
Resolving s3.us-east-1.amazonaws.com (s3.us-east-1.amazonaws.com)... 16.15.178.219, 54.231.140.80, 52.217.133.16, ...
Connecting to s3.us-east-1.amazonaws.com (s3.us-east-1.amazonaws.com)|16.15.178.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4774 (4.7K) [application/json]
Saving to: ‘instruments.json’

instruments.json    100%[===================>]   4.66K  --.-KB/s    in 0s      

2025-06-30 04:14:47 (182 MB/s) - ‘instruments.json’ saved [4774/4774]

--2025-06-30 04:14:47--  https://s3.us-east-1.amazonaws.com/data.heatonresearch.com/bitphonic/Popcorn.json
Resolving s3.us-east-1.amazonaws.com (s3.us-east-1.amazonaws.com)... 16.15.178.219, 54.231.140.80, 52.217.133.16, ...
Connecting to s3.us-east-1.amazonaws.com (s3.us-east-1.amazonaws.com)|16.15.178.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394696 (385K) [a

# Generate WAV for Song

This will generate a "wav" file of the song you specify. This program requires the notes to be saved in a .json, later in this notebook you will see how to convert a MIDI file to this form.

In [3]:
import numpy as np
import wave
import json
import os
import re
from tqdm import tqdm

# choose which JSON‐encoded song to render
SONG_NAME = "Popcorn.json"

# hard‐coded sample rate
sr = 44100

# load all instrument configs
with open(os.path.join(PATH, "instruments.json"), "r") as f:
    instruments = json.load(f)

# timing constants
TICKS_PER_128TH = 1
TICKS_PER_QUARTER = 16
TEMPO_BPM = 120
SECONDS_PER_TICK = 60 / (TEMPO_BPM * TICKS_PER_QUARTER)

# note → frequency helper
NOTE_BASE = {'C': 0, 'D': 2, 'E': 4, 'F': 5, 'G': 7, 'A': 9, 'B': 11}
def note_to_freq(note: str) -> float:
    m = re.match(r'^([A-G])([#b]?)(-?\d+)$', note)
    if not m:
        raise ValueError(f"Bad note format: {note!r}")
    L, acc, octv = m.group(1), m.group(2), int(m.group(3))
    sem = NOTE_BASE[L] + (1 if acc=='#' else -1 if acc=='b' else 0)
    midi = sem + (octv + 1)*12
    return 440.0 * 2 ** ((midi - 69) / 12)

# ADSR envelope generator
def adsr_envelope(t, attack_time, decay_time, sustain_level, release_time):
    N = len(t)
    A = min(int(sr * attack_time), N)
    D = min(A + int(sr * decay_time), N)
    R = min(int(sr * release_time), N)
    release_start = max(N - R, D)

    env = np.zeros_like(t)
    if A > 0:
        env[:A] = np.linspace(0, 1, A)
    if D > A:
        env[A:D] = np.linspace(1, sustain_level, D - A)
    if release_start > D:
        env[D:release_start] = sustain_level
    if N > release_start:
        env[release_start:] = np.linspace(sustain_level, 0, N - release_start)
    return env

# waveform generator
def waveform(kind, freq, t):
    phase = (freq * t) % 1
    if kind == "sine":
        return np.sin(2 * np.pi * freq * t)
    elif kind == "square":
        return np.where(np.sin(2 * np.pi * freq * t) >= 0, 1.0, -1.0)
    elif kind == "triangle":
        return 2 * np.abs(2*phase - 1) - 1
    elif kind == "sawtooth":
        return 2 * phase - 1
    elif kind == "noise":
        return np.random.uniform(-1.0, 1.0, size=len(t))
    else:
        return np.sin(2 * np.pi * freq * t)

# simple 1-pole resonant low-pass (“Mini-Moog style”)
def moog_filter(x, cutoff, resonance):
    y = np.zeros_like(x)
    y1 = y2 = 0.0
    dt = 1.0 / sr
    RC = 1.0 / (2 * np.pi * cutoff)
    alpha = dt / (RC + dt)
    for i, xi in enumerate(x):
        y0 = y1 + alpha * (xi - y1 + resonance * (y1 - y2))
        y[i] = y0
        y2, y1 = y1, y0
    return y

# load multitrack JSON
with open(os.path.join(PATH, SONG_NAME), "r") as f:
    data = json.load(f)
tracks = data.get('tracks', [])

# compute total length
total_ticks = 0
for track in tracks:
    for ev in track['sequence']:
        max_dur = max(n['duration'] for n in ev['notes'])
        total_ticks = max(total_ticks, ev['tick'] + max_dur)

total_seconds = total_ticks * SECONDS_PER_TICK
total_samples = int(total_seconds * sr)

# create buffer
song = np.memmap("song_buffer.dat", dtype=np.float32, mode="w+", shape=(total_samples,))

# render each track separately, using its own synth config
for track in tracks:
    synth_cfg = instruments[track.get("synth", "")]
    adsr = synth_cfg["adsr"]
    attack_time   = adsr["attack_time"]
    decay_time    = adsr["decay_time"]
    sustain_level = adsr["sustain_level"]
    release_time  = adsr["release_time"]

    is_drum_track = track.get("synth") == "drum"
    drum_map = synth_cfg.get("drum_map", {}) if is_drum_track else {}

    lfo_cfg = synth_cfg.get("lfo", {})
    lfo_target = lfo_cfg.get("target")
    lfo_waveform = lfo_cfg.get("waveform", "sine")
    lfo_freq = lfo_cfg.get("freq", 5.0)
    lfo_depth = lfo_cfg.get("depth", 0.01)

    for event in track['sequence']:
        start_tick = event['tick']
        for note_info in event['notes']:
            note_label = note_info['note']

            if is_drum_track and note_label.isdigit():
                drum_spec = drum_map.get(note_label, {
                    "waveform": "noise",
                    "freq": 4000
                })
                waveform_kind = drum_spec["waveform"]
                freq = drum_spec["freq"]

                dur_samps = int(note_info['duration'] * SECONDS_PER_TICK * sr)
                start_samp = int(start_tick * SECONDS_PER_TICK * sr)
                t = np.linspace(0, dur_samps/sr, dur_samps, endpoint=False)

                # Apply amp LFO if needed
                wave = waveform(waveform_kind, freq, t)
                if lfo_target == "amp":
                    amp_lfo = waveform(lfo_waveform, lfo_freq, t)
                    wave *= 1.0 + lfo_depth * amp_lfo

                env = adsr_envelope(t, attack_time, decay_time, sustain_level, release_time)
                chunk = wave * env

            else:
                try:
                    freq = note_to_freq(note_label)
                except ValueError:
                    print(f"Warning: note {note_label!r} not recognized, skipping.")
                    continue

                dur_samps = int(note_info['duration'] * SECONDS_PER_TICK * sr)
                start_samp = int(start_tick * SECONDS_PER_TICK * sr)
                t = np.linspace(0, dur_samps/sr, dur_samps, endpoint=False)

                if lfo_target == "pitch":
                    pitch_lfo = waveform(lfo_waveform, lfo_freq, t)
                    mod_freq = freq * (1.0 + lfo_depth * pitch_lfo)
                else:
                    mod_freq = freq

                amp_lfo = None
                if lfo_target == "amp":
                    amp_lfo = waveform(lfo_waveform, lfo_freq, t)

                wave_sum = np.zeros_like(t)
                for osc in synth_cfg["oscillators"]:
                    f = mod_freq * osc["detune"]
                    sig = waveform(osc["waveform"], f, t)
                    sig = moog_filter(sig,
                                      osc["filter"]["cutoff"],
                                      osc["filter"]["resonance"])
                    wave_sum += sig

                wave_sum /= len(synth_cfg["oscillators"])
                if amp_lfo is not None:
                    wave_sum *= 1.0 + lfo_depth * amp_lfo

                env = adsr_envelope(t, attack_time, decay_time, sustain_level, release_time)
                chunk = wave_sum * env

            end_samp = start_samp + len(chunk)
            if end_samp > total_samples:
                chunk = chunk[: total_samples - start_samp]
            song[start_samp:end_samp] += chunk


# normalization in 10-second chunks
chunk_size = sr * 10
max_val = 0.0
for start in tqdm(range(0, total_samples, chunk_size), desc="Finding max", unit="chunk"):
    end = min(start + chunk_size, total_samples)
    max_val = max(max_val, np.max(np.abs(song[start:end])))

scale = 0.9 / max_val
for start in tqdm(range(0, total_samples, chunk_size), desc="Normalizing", unit="chunk"):
    end = min(start + chunk_size, total_samples)
    song[start:end] *= scale

# write WAV
base, _ = os.path.splitext(SONG_NAME)
wav_path = os.path.join(PATH, f"{base}.wav")

with wave.open(wav_path, "wb") as wf:
    wf.setnchannels(1)
    wf.setsampwidth(2)  # 16-bit
    wf.setframerate(sr)
    for start in tqdm(range(0, total_samples, chunk_size), desc="Writing WAV", unit="chunk"):
        end = min(start + chunk_size, total_samples)
        int_chunk = (song[start:end] * 32767).astype(np.int16)
        wf.writeframes(int_chunk.tobytes())

print(f"Wrote {wav_path}")


Writing WAV: 100%|██████████| 13/13 [00:00<00:00, 574.15chunk/s]

Wrote /content/Popcorn.wav


You can download your wav file.

In [4]:
from google.colab import files
files.download(wav_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

You can also convert the wav to a mp3.

In [43]:
from pathlib import Path
from google.colab import files

wav_path = Path("Popcorn.wav")
mp3_path = wav_path.with_suffix(".mp3")

!ffmpeg -y -i {wav_path} -codec:a libmp3lame -qscale:a 2 {mp3_path}
files.download(mp3_path)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Extract MIDI

This code extracts notes from a MIDI file and stores them to the JSON format that bitphonic uses. You will likely need to map unknown instruments in the synth_map. You can also create new instruments.

If you need to upload a file, use this.

In [ ]:
from google.colab import files
uploaded = files.upload()

Then you can convert. You might also wish to use gdrive.

In [ ]:
import os
import json
import pretty_midi
from pprint import pprint

# ————— CONFIG —————
midi_name = 'Popcorn.mid'

# Tick & tempo settings
TICKS_PER_QUARTER = 16
TEMPO_BPM = 120
SECONDS_PER_TICK = 60.0 / (TEMPO_BPM * TICKS_PER_QUARTER)

# Recognized keyboard instruments (General MIDI program numbers)
KEYBOARD_PROGRAMS = set(range(0, 8)) | set(range(16, 24)) | set(range(80, 88))

synth_map = {
    "Grand Piano": "grand-piano",
    "Piano": "grand-piano",
    "Electric Piano": "electric-piano",
    "Harpsichord": "harpsichord",
    "Drum Kit": "drum",
    "Saxophone": "saxophone",
    "": "grand-piano"
}

def note_name_from_number(n):
    NOTE_NAMES_SHARP = ['C', 'C#', 'D', 'D#', 'E', 'F',
                        'F#', 'G', 'G#', 'A', 'A#', 'B']
    octave = n // 12 - 1
    name = NOTE_NAMES_SHARP[n % 12]
    return f"{name}{octave}"

def midi_to_multitrack_json(midi_path):
    midi = pretty_midi.PrettyMIDI(midi_path)
    print(f"Loaded MIDI: {os.path.basename(midi_path)}")

    all_tracks = []
    for idx, instrument in enumerate(midi.instruments):
        events = []
        for note in instrument.notes:
            start_tick = int(note.start / SECONDS_PER_TICK)
            duration_tick = int((note.end - note.start) / SECONDS_PER_TICK)

            if instrument.is_drum:
                note_label = str(note.pitch)  # e.g., "38" for snare
            else:
                note_label = note_name_from_number(note.pitch)

            events.append({
                'tick': start_tick,
                'notes': [{
                    'note': note_label,
                    'duration': duration_tick
                }]
            })

        # Merge chords
        tick_map = {}
        for ev in events:
            tick_map.setdefault(int(ev['tick']), []).extend(ev['notes'])

        sequence = [{'tick': tick, 'notes': notes} for tick, notes in sorted(tick_map.items())]

        if instrument.name not in synth_map:
            raise ValueError(f"Unmapped instrument: {instrument.name}")

        all_tracks.append({
            'synth': synth_map[instrument.name],
            'name': instrument.name or f"track_{idx}",
            'program': int(instrument.program),
            'is_drum': bool(instrument.is_drum),
            'sequence': sequence
        })

    return all_tracks


# ————— MAIN —————
midi_path = os.path.join(PATH, midi_name)
tracks = midi_to_multitrack_json(midi_path)

json_path = os.path.splitext(midi_path)[0] + '.json'
with open(json_path, 'w') as f:
    json.dump({'tracks': tracks}, f, indent=2)

print(f"Wrote multitrack JSON to {json_path}\n")



Loaded MIDI: rush_e_real.mid
Wrote multitrack JSON to /content/drive/MyDrive/data/music/rush_e_real.json

